In [1]:
import pandas as pd
import numpy as np
import os
import polars as pl

### Case study for Zip code 75212

Workflow:
- Grab all hospitalization records for the zip code 

In [8]:
hospital_data = "D:/GitHub/THCICAnalysis/Hospital/"
root = "../CAMS/"
units = "../assets/CAMS_units.csv"
zip_2010 = "../assets/tx_texas_zip_codes_geo.min.json"
# hospital_data = "/media/teamlary/ssd/Discharge Data/Inpatient/Data/"
hospital_data = "D:/GitHub/THCICAnalysis/Hospital/"
root = "../CAMS/"
census_dir = "../Census/"
icd_data = "../icd10/"

zipPop = "../assets/census/interpolatedPopulations.csv"

full_zip_population = pl.read_csv(zipPop)
full_zip_population

PAT_ZIP,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
73949,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1764,1662,1561,1459,1357
75001,8175,8569,8963,9357,9751,10145,10540,10934,11328,11722,12116,12510,12892,13274,13657,14039,14421,14794,15167,15541,15914,16287
75002,37171,39444,41717,43990,46263,48536,50810,53083,55356,57629,59902,62175,63513,64850,66188,67525,68863,69311,69759,70206,70654,71102
75006,45990,45985,45979,45974,45969,45963,45958,45952,45947,45942,45936,45931,46752,47574,48395,49217,50038,49651,49264,48878,48491,48104
75007,50357,50408,50460,50511,50562,50613,50665,50716,50767,50818,50870,50921,51599,52277,52956,53634,54312,54339,54366,54394,54421,54448
75009,4913,5232,5550,5869,6188,6506,6825,7143,7462,7781,8099,8418,8778,9139,9499,9860,10220,13123,16027,18930,21834,24737
75010,13141,14015,14889,15763,16637,17511,18386,19260,20134,21008,21882,22756,23352,23949,24545,25142,25738,26616,27494,28373,29251,30129
75013,11584,13082,14579,16077,17575,19073,20570,22068,23566,25064,26561,28059,29957,31855,33752,35650,37548,39000,40452,41903,43355,44807
75019,35805,36077,36349,36621,36893,37165,37437,37709,37981,38253,38525,38797,39210,39623,40037,40450,40863,41555,42247,42940,43632,44324


In [4]:
icd = pd.read_pickle('../assets/ICDPickle.pkl')
icd = pl.DataFrame._from_pandas(icd)
icd

PRINC_DIAG_CODE,Count,ICD-9,icd9cm_mult
str,f64,str,list[str]
"""Z3800""",3.940227e6,"""V3900""","[""V3000"", ""V3900""]"
"""Z3801""",2.099687e6,"""V3001""","[""V3001""]"
"""A419""",1.211138e6,"""99591""","[""0389"", ""99591""]"
"""J189""",820711.0,"""486""","[""486""]"
"""Z5189""",676973.0,"""V589""","[""V5889"", ""V589""]"
"""O3421""",627506.0,"""65423""","[""65421"", ""65423""]"
"""J441""",500761.0,"""49322""","[""49121"", ""49322""]"
"""I2510""",488267.0,"""4292""","[""41401"", ""4292""]"
"""N179""",470121.0,"""5849""","[""5849""]"


In [6]:
start_year = 2005
end_year = 2022

hospital_quarters = [f"{quarter}q{year}" for year in range(start_year, end_year + 1) for quarter in range(1, 5) if not (year == end_year and quarter > 2)]
hospital_quarters = hospital_quarters[:-1]
icd9_subset = hospital_quarters[hospital_quarters.index(f'1q{start_year}'):hospital_quarters.index('4q2015')]
icd10_subset = hospital_quarters[hospital_quarters.index('4q2015'):hospital_quarters.index(f'1q{end_year}')]

In [16]:
def loopICDPerQuarter(time_period, nthresh=1, icd10=True):
    data = pl.DataFrame()
    for ind, quarter in enumerate(time_period):
        print(quarter)
        hospital_df = pl.read_csv(f"{hospital_data}PUDF_base1_{quarter}_tab.csv", infer_schema_length=0)
        hospital_df = (hospital_df
                        # .filter(~(pl.col('PAT_ZIP').str.ends_with('.0')))
                        .with_columns(pl.col('PAT_ZIP').str.replace(r'\.0$',''))
                        .filter(pl.col('PAT_ZIP') != '`')
                        .filter(pl.col('PAT_ZIP').str.lengths() == 5)
                        .filter(pl.col('PAT_ZIP') == '75212')
                        .with_columns(pl.col('PAT_ZIP').cast(pl.Int64))
                        .select(['SEX_CODE','PAT_ZIP','ADMIT_WEEKDAY','LENGTH_OF_STAY','PAT_AGE','RACE','ETHNICITY','TOTAL_CHARGES','PRINC_DIAG_CODE'])
                        
        )

        data = pl.concat([data, hospital_df])
        # print(hospital_df.glimpse())
        # zip_population = interpCensus(quarter)
        # year = quarter[2:]
        # zip_population = (full_zip_population
        #                   .select(['PAT_ZIP',year])
        #                   .rename({year: 'population'})
        # )
        # # print(zip_population.glimpse())

        # if icd10 == True:
        #     codes = icd['PRINC_DIAG_CODE']
        # else:
        #     codes = icd['ICD-9']

        # for sind, icd_code in enumerate(codes[:500]):
        #     os.makedirs(f'../icd10_polars/{icd["PRINC_DIAG_CODE"][sind]}', exist_ok=True)
        #     # print(icd_code)
        #     # if icd_code != 'N210':
        #     #     continue

        #     base_df = pl.DataFrame()
        #     if icd10 == False:
        #         for item in icd['icd9cm_mult'][sind]: 
        #             # print(icd_code)
        #             icd_df = (hospital_df
        #                       .filter(hospital_df['PRINC_DIAG_CODE'] == item)
        #                       .group_by(['PAT_ZIP']).count()                              
        #                       .rename({'count': 'ICD'})
        #                       .join(zip_population, on='PAT_ZIP')
        #                       .with_columns((pl.col('ICD')/pl.col('population'))
        #                                     .alias('normalized'))                        
        #                       .filter(pl.col('ICD') >= nthresh)
        #                       .sort('ICD', descending=True)
        #             )
        #             # print(icd_df.glimpse())

        #             base_df = pl.concat([base_df, icd_df])
        #     else:
        #         icd_df = (hospital_df
        #                       .filter(hospital_df['PRINC_DIAG_CODE'] == icd_code)
        #                       .group_by(['PAT_ZIP']).count()
        #                       .sort('count', descending=True)
        #                       .rename({'count': 'ICD'})
        #                       .join(zip_population, on='PAT_ZIP')
        #                       .with_columns((pl.col('ICD')/pl.col('population'))
        #                                     .alias('normalized'))                              
        #                       .filter(pl.col('ICD') >= nthresh) 
        #             )
        #         #print(icd_df.glimpse())
        #         base_df = pl.concat([base_df, icd_df])
        #     save_quarter = quarter[2:] + 'q' + quarter[:1]
        #     #print(save_quarter)
        #     base_df.write_csv(f'../icd10_polars/{icd["PRINC_DIAG_CODE"][sind]}/{save_quarter}.csv')
        
            # del hospital_df
    del hospital_df

    return data

In [21]:
# icd9_hospitalizations = loopICDPerQuarter(icd9_subset, icd10=False)
icd10_hospitalizations = loopICDPerQuarter(icd10_subset, icd10=True)

4q2015
1q2016
2q2016
3q2016
4q2016
1q2017
2q2017
3q2017
4q2017
1q2018
2q2018
3q2018
4q2018
1q2019
2q2019
3q2019
4q2019
1q2020
2q2020
3q2020
4q2020
1q2021
2q2021
3q2021
4q2021


In [20]:
icd9_hospitalizations

SEX_CODE,PAT_ZIP,ADMIT_WEEKDAY,LENGTH_OF_STAY,PAT_AGE,RACE,ETHNICITY,TOTAL_CHARGES,PRINC_DIAG_CODE
str,i64,str,str,str,str,str,str,str
"""F""",75212,"""5""","""49.0""","""6""","""3""","""2.0""","""21952.0""","""29590"""
"""F""",75212,"""2""","""139.0""","""11""","""3""","""2.0""","""6227.2""","""29590"""
"""F""",75212,"""4""","""61.0""","""12""","""3""","""2.0""","""26432.0""","""29570"""
"""M""",75212,"""2""","""7.0""","""10""","""3""","""2.0""","""3136.0""","""29680"""
"""M""",75212,"""2""","""22.0""","""9""","""3""","""2.0""","""9856.0""","""29510"""
"""M""",75212,"""1""","""3.0""","""10""","""3""","""2.0""","""1344.0""","""29590"""
"""M""",75212,"""4""","""148.0""","""16""","""3""","""2.0""","""6630.4""","""29530"""
"""M""",75212,"""3""","""131.0""","""12""","""3""","""2.0""","""6052.2""","""29510"""
"""F""",75212,"""7""","""2.0""","""9""","""4""","""1.0""","""3031.1""","""66041"""


In [23]:
hospitalizations = pl.concat([icd9_hospitalizations, icd10_hospitalizations])
hospitalizations

SEX_CODE,PAT_ZIP,ADMIT_WEEKDAY,LENGTH_OF_STAY,PAT_AGE,RACE,ETHNICITY,TOTAL_CHARGES,PRINC_DIAG_CODE
str,i64,str,str,str,str,str,str,str
"""F""",75212,"""5""","""49.0""","""6""","""3""","""2.0""","""21952.0""","""29590"""
"""F""",75212,"""2""","""139.0""","""11""","""3""","""2.0""","""6227.2""","""29590"""
"""F""",75212,"""4""","""61.0""","""12""","""3""","""2.0""","""26432.0""","""29570"""
"""M""",75212,"""2""","""7.0""","""10""","""3""","""2.0""","""3136.0""","""29680"""
"""M""",75212,"""2""","""22.0""","""9""","""3""","""2.0""","""9856.0""","""29510"""
"""M""",75212,"""1""","""3.0""","""10""","""3""","""2.0""","""1344.0""","""29590"""
"""M""",75212,"""4""","""148.0""","""16""","""3""","""2.0""","""6630.4""","""29530"""
"""M""",75212,"""3""","""131.0""","""12""","""3""","""2.0""","""6052.2""","""29510"""
"""F""",75212,"""7""","""2.0""","""9""","""4""","""1.0""","""3031.1""","""66041"""


In [37]:
icd_df = icd.to_pandas()
icd_df

,PRINC_DIAG_CODE,Count,ICD-9,icd9cm_mult
0,Z3800,3940227.0,V3900,"[V3000, V3900]"
1,Z3801,2099687.0,V3001,[V3001]
2,A419,1211138.0,99591,"[0389, 99591]"
3,J189,820711.0,486,[486]
4,Z5189,676973.0,V589,"[V5889, V589]"
...,...,...,...,...
6222,A360,2.0,0320,[0320]
6223,A240,2.0,024,[024]
6224,Z569,2.0,V621,[V621]
6225,L250,2.0,69281,[69281]


In [38]:
icd9_df = icd9_hospitalizations.to_pandas()
icd10_df = icd10_hospitalizations.to_pandas()

In [41]:
icd_df

,PRINC_DIAG_CODE,Count,ICD-9,icd9cm_mult
0,Z3800,3940227.0,V3900,"[V3000, V3900]"
1,Z3801,2099687.0,V3001,[V3001]
2,A419,1211138.0,99591,"[0389, 99591]"
3,J189,820711.0,486,[486]
4,Z5189,676973.0,V589,"[V5889, V589]"
...,...,...,...,...
6222,A360,2.0,0320,[0320]
6223,A240,2.0,024,[024]
6224,Z569,2.0,V621,[V621]
6225,L250,2.0,69281,[69281]


In [51]:
len([i for i in icd_df['icd9cm_mult'] if len(i) == 2])

796

In [55]:
dummy_list = [[1,2,3,4],[5,6]]


In [61]:
[item for sublist in dummy_list for item in sublist]

[1, 2, 3, 4, 5, 6]

In [62]:
[j for i  in dummy_list for j in i] 

[1, 2, 3, 4, 5, 6]

In [54]:
[list(i) for i in icd_df['icd9cm_mult']]

[['V3000', 'V3900'],
 ['V3001'],
 ['0389', '99591'],
 ['486'],
 ['V5889', 'V589'],
 ['65421', '65423'],
 ['49121', '49322'],
 ['41401', '4292'],
 ['5849'],
 ['41071'],
 ['5990'],
 ['650'],
 ['66411', '66414'],
 ['66401', '66404'],
 ['71536', '71596'],
 ['4280', '4289'],
 ['64511', '64513'],
 ['42731'],
 ['65971'],
 ['78659'],
 ['43491'],
 ['64891'],
 ['29633'],
 ['40401', '40411', '40491'],
 ['V5811'],
 ['27801'],
 ['25080'],
 ['27651'],
 ['51881'],
 ['40201', '40211', '40291'],
 ['5070'],
 ['56211'],
 ['5770'],
 ['57400'],
 ['64421'],
 ['5589'],
 ['99859'],
 ['41519'],
 ['V3101', 'V3201', 'V3301'],
 ['4280', '42823'],
 ['5409'],
 ['29570'],
 ['7802'],
 ['49392'],
 ['03842', '99591'],
 ['65811'],
 ['42781', '42789'],
 ['29680'],
 ['78650'],
 ['2189'],
 ['5789'],
 ['66331', '66381'],
 ['40301', '40311', '40391'],
 ['5609'],
 ['29620', '311'],
 ['49122', '49321'],
 ['4280', '42833'],
 ['29690'],
 ['0388', '99591'],
 ['2761'],
 ['25080'],
 ['4359'],
 ['65801', '65803'],
 ['64821'],
 ['642

In [ ]:
def icd9toicd10(row):
    for entry in 

In [43]:
icd9_df

,SEX_CODE,PAT_ZIP,ADMIT_WEEKDAY,LENGTH_OF_STAY,PAT_AGE,RACE,ETHNICITY,TOTAL_CHARGES,PRINC_DIAG_CODE
0,F,75212,5,49.0,6,3,2.0,21952.0,29590
1,F,75212,2,139.0,11,3,2.0,6227.2,29590
2,F,75212,4,61.0,12,3,2.0,26432.0,29570
3,M,75212,2,7.0,10,3,2.0,3136.0,29680
4,M,75212,2,22.0,9,3,2.0,9856.0,29510
...,...,...,...,...,...,...,...,...,...
35358,M,75212,3.0,23.0,17,2,2.0,37598.74,25080
35359,M,75212,3.0,22.0,12,4,2,100928.65,V5844
35360,M,75212,4.0,6.0,7,3,2,9000.0,2989
35361,M,75212,1.0,4.0,15,3,2,5100.0,29530


In [30]:
hospital_df = hospitalizations.to_pandas()
hospital_df

,SEX_CODE,PAT_ZIP,ADMIT_WEEKDAY,LENGTH_OF_STAY,PAT_AGE,RACE,ETHNICITY,TOTAL_CHARGES,PRINC_DIAG_CODE
0,F,75212,5,49.0,6,3,2.0,21952.0,29590
1,F,75212,2,139.0,11,3,2.0,6227.2,29590
2,F,75212,4,61.0,12,3,2.0,26432.0,29570
3,M,75212,2,7.0,10,3,2.0,3136.0,29680
4,M,75212,2,22.0,9,3,2.0,9856.0,29510
...,...,...,...,...,...,...,...,...,...
52914,F,75212,5,3,15,3,2,23670.96,K5732
52915,M,75212,5,2,17,3,2,11879.94,N3090
52916,M,75212,7,10,7,4,1,100848.1,J690
52917,F,75212,1,4,6,3,2,40421.15,A4101


In [35]:
hospital_df.groupby('PRINC_DIAG_CODE')['PAT_ZIP'].count().sort_values(ascending=False)[:20]

PRINC_DIAG_CODE
V3000    4210
Z3800    1848
V3001    1489
A419      906
Z3801     771
65421     697
486       612
0389      517
66401     507
V5789     468
64511     446
5849      439
4280      415
66411     390
650       381
I130      376
5990      374
42823     305
43491     299
41071     287
Name: PAT_ZIP, dtype: int64